In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## moving the data 

**이것을 이제 module화 시켜서, 다양한 tar file, self.glob에 대해서 해서 여러개의 file들을 평행하게 (using multiproceissing or sth) 만들어서 돌리기!)**

In [5]:
import tarfile
import os 
import shutil
import glob 
import subprocess

class Args():
    def __init__(self,T1_DWI_dir, T2_dir, save_dir, glob):
        self.T1_DWI_dir = T1_DWI_dir
        self.T2_dir = T2_dir
        self.save_dir = save_dir
        self.glob = glob
        

In [10]:
args = Args(T1_DWI_dir = "/scratch/08834/tg881334/ABCD_preproc/1.data/T1_DWI/NDARINV0.tar",
           T2_dir = "/scratch/08834/tg881334/ABCD_preproc/1.data/T2/NDARINV0-5.tar",
           save_dir = "/scratch/08834/tg881334/ABCD_preproc/2.BIDS/",
           glob = "NDARINV0A")

In [11]:

class tar_ABCD():
    def __init__(self, args, do="T1_DTI"):
        """
        do  : "T1_DTI" or "T2"
        """
        if do == "T1_DTI":
            self.tar_dir = args.T1_DWI_dir
        elif do == "T2":
            self.tar_dir = args.T2_dir
        else : 
            raise ValueError("not one of the modalities that can be done")
            
            
        self.file_to_untar = []
        self.last_member_names = []
   

    def create_list(self):

        tar = tarfile.open(self.tar_dir)
        for i,member in enumerate(tar):
            file_name = member.name.split('/')[-1]
            if not member.isfile(): #skip if not file
                print("NOT", member)
                continue 
            elif member.name.split('_')[-3] == "baselineYear1Arm1": #skip if baseline
                continue 
            elif args.glob not in file_name: #skip if doesn't match glob 
                continue 
            
            self.file_to_untar.append(member)
            
            #if i > 1000:
            #    break
        
        self.last_member_names.append(member)
        
        #self.file_to_untar = file_to_untar
        #self.last_member_names = last_member_names
        self.tar = tar
        print(self.tar_dir)                    

    def extract_move(self):
        print(self.file_to_untar)
        self.tar.extractall(members = self.file_to_untar, path = args.save_dir) #extract files
        for i,member in enumerate(self.last_member_names):
            #raise ValueError("muyst be changed")
            upper_dir = os.path.split(member.name)[0] #get uppder dir name
            
            [shutil.move(file, args.save_dir) for file in glob.glob(os.path.join(args.save_dir, f"{upper_dir}/*"))] #move them to the actual directory I wanted
                
    def untar_BIDS(self):
        os.chdir(args.save_dir)

        #unzip the .tgz files
        for file in os.listdir(args.save_dir):
            print(file)
            if "tgz" in file: #i.e. if tgz
                subprocess.run(f'tar zxvf {file}', shell = True)
                subprocess.run(f'rm {file}', shell = True)
            elif not "sub-" in file and "dataset_description.json" != file: #i.e. 이미 unzip된 sub-파일이 아니라면, 또는 dataset_description이 아니라면 
                shutil.rmtree(file) #remove the nuisance stuff
        
        

In [12]:
a = tar_ABCD(args, do = "T1_DTI")
a.create_list()
a.extract_move()
a.untar_BIDS()

b = tar_ABCD(args, do = "T2")
b.create_list()
b.extract_move()
b.untar_BIDS()

NOT <TarInfo 'NDARINV0' at 0x2ab440b19900>
/scratch/08834/tg881334/ABCD_preproc/1.data/T1_DWI/NDARINV0.tar
[<TarInfo 'NDARINV0/NDARINV0AHJHA5D_2YearFollowUpYArm1_ABCD-MPROC-T1_20200204134321.tgz' at 0x2ab440b1a800>, <TarInfo 'NDARINV0/NDARINV0A9K5L4R_2YearFollowUpYArm1_ABCD-MPROC-T1_20190507124028.tgz' at 0x2ab440cbca00>, <TarInfo 'NDARINV0/NDARINV0AUBJJJ4_2YearFollowUpYArm1_ABCD-MPROC-DTI_20200215121249.tgz' at 0x2ab440cbd6c0>, <TarInfo 'NDARINV0/NDARINV0AUBJJJ4_2YearFollowUpYArm1_ABCD-MPROC-DTI_20200215121811.tgz' at 0x2ab440bf5b40>, <TarInfo 'NDARINV0/NDARINV0A6WVRZY_2YearFollowUpYArm1_ABCD-MPROC-DTI_20190708132736.tgz' at 0x2ab440f8a800>, <TarInfo 'NDARINV0/NDARINV0AZFZ65C_2YearFollowUpYArm1_ABCD-MPROC-DTI_20200224171107.tgz' at 0x2ab440f925c0>, <TarInfo 'NDARINV0/NDARINV0A9K5L4R_2YearFollowUpYArm1_ABCD-MPROC-DTI_20190507125713.tgz' at 0x2ab440f93580>, <TarInfo 'NDARINV0/NDARINV0AUBJJJ4_2YearFollowUpYArm1_ABCD-MPROC-T1_20200215114915.tgz' at 0x2ab440f93880>, <TarInfo 'NDARINV0/NDAR

tar: sub-NDARINV0AUBJJJ4/ses-2YearFollowUpYArm1/dwi/sub-NDARINV0AUBJJJ4_ses-2YearFollowUpYArm1_run-02_dwi.bval: implausibly old time stamp 1969-12-31 18:00:00
tar: sub-NDARINV0AUBJJJ4/ses-2YearFollowUpYArm1/dwi/sub-NDARINV0AUBJJJ4_ses-2YearFollowUpYArm1_run-02_dwi.bvec: implausibly old time stamp 1969-12-31 18:00:00
tar: sub-NDARINV0AUBJJJ4/ses-2YearFollowUpYArm1/dwi/sub-NDARINV0AUBJJJ4_ses-2YearFollowUpYArm1_run-02_dwi.json: implausibly old time stamp 1969-12-31 18:00:00


NDARINV0
NDARINV0A6WVRZY_2YearFollowUpYArm1_ABCD-MPROC-T1_20190708130607.tgz
sub-NDARINV0A6WVRZY/ses-2YearFollowUpYArm1/anat/sub-NDARINV0A6WVRZY_ses-2YearFollowUpYArm1_run-01_T1w.nii
dataset_description.json
sub-NDARINV0A6WVRZY/ses-2YearFollowUpYArm1/anat/sub-NDARINV0A6WVRZY_ses-2YearFollowUpYArm1_run-01_T1w.json
NDARINV0A4P0LWM_2YearFollowUpYArm1_ABCD-MPROC-DTI_20200229133342.tgz
sub-NDARINV0A4P0LWM/ses-2YearFollowUpYArm1/dwi/sub-NDARINV0A4P0LWM_ses-2YearFollowUpYArm1_run-01_dwi.nii


tar: sub-NDARINV0A6WVRZY/ses-2YearFollowUpYArm1/anat/sub-NDARINV0A6WVRZY_ses-2YearFollowUpYArm1_run-01_T1w.json: implausibly old time stamp 1969-12-31 18:00:00


dataset_description.json
sub-NDARINV0A4P0LWM/ses-2YearFollowUpYArm1/dwi/sub-NDARINV0A4P0LWM_ses-2YearFollowUpYArm1_run-01_dwi.bval
sub-NDARINV0A4P0LWM/ses-2YearFollowUpYArm1/dwi/sub-NDARINV0A4P0LWM_ses-2YearFollowUpYArm1_run-01_dwi.bvec
sub-NDARINV0A4P0LWM/ses-2YearFollowUpYArm1/dwi/sub-NDARINV0A4P0LWM_ses-2YearFollowUpYArm1_run-01_dwi.json


tar: sub-NDARINV0A4P0LWM/ses-2YearFollowUpYArm1/dwi/sub-NDARINV0A4P0LWM_ses-2YearFollowUpYArm1_run-01_dwi.bval: implausibly old time stamp 1969-12-31 18:00:00
tar: sub-NDARINV0A4P0LWM/ses-2YearFollowUpYArm1/dwi/sub-NDARINV0A4P0LWM_ses-2YearFollowUpYArm1_run-01_dwi.bvec: implausibly old time stamp 1969-12-31 18:00:00
tar: sub-NDARINV0A4P0LWM/ses-2YearFollowUpYArm1/dwi/sub-NDARINV0A4P0LWM_ses-2YearFollowUpYArm1_run-01_dwi.json: implausibly old time stamp 1969-12-31 18:00:00


NDARINV0AUBJJJ4_2YearFollowUpYArm1_ABCD-MPROC-T1_20200215114915.tgz
sub-NDARINV0AUBJJJ4/ses-2YearFollowUpYArm1/anat/sub-NDARINV0AUBJJJ4_ses-2YearFollowUpYArm1_run-01_T1w.nii


KeyboardInterrupt: 

In [14]:
T1_DWI_dir = "/scratch/08834/tg881334/ABCD_preproc/1.data/T1_DWI/NDARINV0.tar"
T2_dir = "/scratch/08834/tg881334/ABCD_preproc/1.data/T2/NDARINV0-5.tar"
save_dir = "/scratch/08834/tg881334/ABCD_preproc/2.BIDS/"
args = Args(T1_DWI_dir = T1_DWI_dir, T2_dir = T2_dir, save_dir = save_dir,
           glob = "NDARINV0AB")

In [ ]:
              
from multiprocessing import Pool
p = Pool(160)


T1_DWI_dir = "/scratch/08834/tg881334/ABCD_preproc/1.data/T1_DWI/NDARINV0.tar"
T2_dir = "/scratch/08834/tg881334/ABCD_preproc/1.data/T2/NDARINV0-5.tar"
save_dir = "/scratch/08834/tg881334/ABCD_preproc/2.BIDS/"
args = Args(T1_DWI_dir = T1_DWI_dir, T2_dir = T2_dir, save_dir = save_dir,
           glob = "NDARINV0AB")
           
arg_list = []
for i in ['0A','0B']:
    args = Args(T1_DWI_dir = T1_DWI_dir, T2_dir = T2_dir, save_dir = save_dir,
           glob = "NDARINV"+i)
    arg_list.append(args)
    print(args.glob)

def untar_parallel(args):
    a = tar_ABCD(args, do = "T1_DTI")
    a.create_list()
    a.extract_move()
    a.untar_BIDS()
    
    b = tar_ABCD(args, do = "T2")
    b.create_list()
    b.extract_move()
    b.untar_BIDS()
    

for s in p.imap(untar_parallel, arg_list):
    print("===============")

NDARINV0A
NDARINV0B


Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home1/08834/tg881334/logout/envs/CHA_preproc/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home1/08834/tg881334/logout/envs/CHA_preproc/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home1/08834/tg881334/logout/envs/CHA_preproc/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home1/08834/tg881334/logout/envs/CHA_preproc/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home1/08834/tg881334/logout/envs/CHA_preproc/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home1/08834/tg881334/logout/envs/CHA_preproc/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home1/088

In [6]:
#remove ses and force it to be moved

In [7]:
os.listdir()

['sub-NDARINV0A4ZDYNL',
 'sub-NDARINV01NAYMZH',
 'sub-NDARINV01AJ15N9',
 'sub-NDARINV014RTM1V',
 'sub-NDARINV0A6WVRZY',
 'sub-NDARINV019DXLU4',
 'sub-NDARINV0AHJHA5D',
 'sub-NDARINV01D03VR7',
 'sub-NDARINV01EN91PG',
 'sub-NDARINV010ZM3H9',
 'dataset_description.json',
 'sub-NDARINV0A4P0LWM',
 'sub-NDARINV01TXCDJN',
 'sub-NDARINV0A9K5L4R',
 'sub-NDARINV01ELX9L6',
 'sub-NDARINV0AAKGYA2',
 'sub-NDARINV01Z8HAPV',
 'sub-NDARINV0AEBMADL',
 'sub-NDARINV0191C80U',
 'sub-NDARINV0APJMRD1',
 'sub-NDARINV01RGTWD2']

In [8]:
exit()

### trying to do things


In [1]:
import tarfile
import os 
import shutil
import glob 
import subprocess

class Args():
    def __init__(self):
        self.T1_DWI_dir = "/scratch/08834/tg881334/NDARINV0.tar"
        self.T2_dir = "/scratch/08834/tg881334/NDARINV0-5.tar"
        self.save_dir = "/scratch/08834/tg881334/2019/2.BIDS"
        self.glob = "NDARINV0X" #"NDARINV0BL" #let's look at this particular case (이러면 원래 너무 많을 수도 있따 )
    

In [2]:
args = Args()

In [3]:
os.listdir(args.save_dir)

['sub-NDARINV0A4ZDYNL',
 'sub-NDARINV0Z5FF0JL',
 'sub-NDARINV0E350J5D',
 'sub-NDARINV0ZHHWCMZ',
 'sub-NDARINV0XDBADBW',
 'sub-NDARINV01NAYMZH',
 'sub-NDARINV01AJ15N9',
 'sub-NDARINV0E9UFT0K',
 'sub-NDARINV0XTVAGV2',
 'sub-NDARINV0EV57VEX',
 'sub-NDARINV014RTM1V',
 'sub-NDARINV0XTAYD1B',
 'sub-NDARINV0A6WVRZY',
 'sub-NDARINV019DXLU4',
 'sub-NDARINV0ZAYELPK',
 'sub-NDARINV0XU7Z6RH',
 'sub-NDARINV0ZTJ2432',
 'sub-NDARINV0YE7L9KU',
 'sub-NDARINV01D03VR7',
 'sub-NDARINV0EXY6KFW',
 'sub-NDARINV01EN91PG',
 'sub-NDARINV0E1RBR9J',
 'sub-NDARINV010ZM3H9',
 'dataset_description.json',
 'sub-NDARINV0A4P0LWM',
 'sub-NDARINV01TXCDJN',
 'sub-NDARINV0A9K5L4R',
 'sub-NDARINV01ELX9L6',
 'sub-NDARINV0ZPPK2ML',
 'sub-NDARINV0XFVW4LF',
 'sub-NDARINV0AAKGYA2',
 'sub-NDARINV01Z8HAPV',
 'sub-NDARINV0YVKYMJX',
 'sub-NDARINV0X02CUCY',
 'sub-NDARINV0X59EG8X',
 'sub-NDARINV0Y9F2PL1',
 'sub-NDARINV0AEBMADL',
 'sub-NDARINV0191C80U',
 'sub-NDARINV0APJMRD1',
 'sub-NDARINV0XKVPBB0',
 'sub-NDARINV0YLLNXRL',
 'sub-NDARI

### below : trash

In [9]:
args = Args()
file_to_untar = []


last_member_names = []
for tar_dir in [args.T1_DWI_dir]: #[args.T1_DWI_dir]:#[args.T2_dir, args.T1_DWI_dir]:
    tar = tarfile.open(tar_dir)
    for i,member in enumerate(tar):
        file_name = member.name.split('/')[-1]
        if not member.isfile(): #skip if not file
            print("NOT", member)
            continue 
        elif member.name.split('_')[-3] == "baselineYear1Arm1": #skip if baseline
            print(member.name)
            continue 
        elif args.glob not in file_name: #skip if doesn't match glob 
            continue 
        
        file_to_untar.append(member)
    
        if i > 100000: #임의의 제한.. 굳이 둘 필요 없음!
            break  
    last_member_names.append(member)
    print(tar_dir)

NOT <TarInfo 'NDARINV0' at 0x2b16a4ec25c0>
NDARINV0/NDARINV007W6H7B_baselineYear1Arm1_ABCD-MPROC-DTI_20170224181739.tgz
NDARINV0/NDARINV04JY6730_baselineYear1Arm1_ABCD-MPROC-T1_20170725163532.tgz
NDARINV0/NDARINV0L3VJZEL_baselineYear1Arm1_ABCD-MPROC-DTI_20170413123753.tgz
NDARINV0/NDARINV0G42TBW1_baselineYear1Arm1_ABCD-MPROC-DTI_20170619152507.tgz
NDARINV0/NDARINV0RB5CTN0_baselineYear1Arm1_ABCD-MPROC-T1_20180829145602.tgz
NDARINV0/NDARINV08K0R9C4_baselineYear1Arm1_ABCD-MPROC-DTI_20170824165607.tgz
NDARINV0/NDARINV0RA4PBPV_baselineYear1Arm1_ABCD-MPROC-T1_20171001125316.tgz
NDARINV0/NDARINV08P3GNDV_baselineYear1Arm1_ABCD-MPROC-DTI_20180219121500.tgz
NDARINV0/NDARINV0DVK13LU_baselineYear1Arm1_ABCD-MPROC-DTI_20170316143433.tgz
NDARINV0/NDARINV0NGG5VLJ_baselineYear1Arm1_ABCD-MPROC-T1_20180616152053.tgz
NDARINV0/NDARINV08R2PTT1_baselineYear1Arm1_ABCD-MPROC-T1_20170419161447.tgz
NDARINV0/NDARINV08GBDG8X_baselineYear1Arm1_ABCD-MPROC-T1_20171018113224.tgz
NDARINV0/NDARINV0314RN9P_baselineYear1A

NDARINV0/NDARINV0YDL0DWV_baselineYear1Arm1_ABCD-MPROC-DTI_20180412193649.tgz
NDARINV0/NDARINV04CLBZAD_baselineYear1Arm1_ABCD-MPROC-DTI_20171207150023.tgz
NDARINV0/NDARINV0KHYZ082_baselineYear1Arm1_ABCD-MPROC-T1_20170909160910.tgz
NDARINV0/NDARINV0PHTY15N_baselineYear1Arm1_ABCD-MPROC-T1_20170827110738.tgz
NDARINV0/NDARINV08WDFUCE_baselineYear1Arm1_ABCD-MPROC-T1_20180711124038.tgz
NDARINV0/NDARINV021N0FLH_baselineYear1Arm1_ABCD-MPROC-T1_20171018144030.tgz
NDARINV0/NDARINV021N0FLH_baselineYear1Arm1_ABCD-MPROC-DTI_20171018145746.tgz
NDARINV0/NDARINV0YLJKU2J_baselineYear1Arm1_ABCD-MPROC-DTI_20180301174807.tgz
NDARINV0/NDARINV0W4PR2WT_baselineYear1Arm1_ABCD-MPROC-DTI_20180716140419.tgz
NDARINV0/NDARINV0PJ81CA5_baselineYear1Arm1_ABCD-MPROC-DTI_20180202120126.tgz
NDARINV0/NDARINV0V139J8T_baselineYear1Arm1_ABCD-MPROC-T1_20170731145311.tgz
NDARINV0/NDARINV0VY70352_baselineYear1Arm1_ABCD-MPROC-T1_20171007152408.tgz
NDARINV0/NDARINV019DXLU4_baselineYear1Arm1_ABCD-MPROC-DTI_20170320133220.tgz
NDARI

NDARINV0/NDARINV04R9REGF_baselineYear1Arm1_ABCD-MPROC-DTI_20180512122702.tgz
NDARINV0/NDARINV08YFFYY2_baselineYear1Arm1_ABCD-MPROC-T1_20170930115010.tgz
NDARINV0/NDARINV01AJ15N9_baselineYear1Arm1_ABCD-MPROC-DTI_20171209095404.tgz
NDARINV0/NDARINV01NAYMZH_baselineYear1Arm1_ABCD-MPROC-T1_20170809113714.tgz
NDARINV0/NDARINV09AEBLZH_baselineYear1Arm1_ABCD-MPROC-DTI_20180119160723.tgz
NDARINV0/NDARINV0J9U5XRJ_baselineYear1Arm1_ABCD-MPROC-T1_20180908153518.tgz
NDARINV0/NDARINV07THBLHG_baselineYear1Arm1_ABCD-MPROC-DTI_20180816142440.tgz
NDARINV0/NDARINV0DJJP33V_baselineYear1Arm1_ABCD-MPROC-DTI_20171117104020.tgz
NDARINV0/NDARINV06DP74KL_baselineYear1Arm1_ABCD-MPROC-T1_20171007111410.tgz
NDARINV0/NDARINV05LGG3GZ_baselineYear1Arm1_ABCD-MPROC-DTI_20171029111145.tgz
NDARINV0/NDARINV0UPVEC1J_baselineYear1Arm1_ABCD-MPROC-DTI_20170318142426.tgz
NDARINV0/NDARINV0ZTJ2432_baselineYear1Arm1_ABCD-MPROC-DTI_20170509190655.tgz
NDARINV0/NDARINV0387TP8R_baselineYear1Arm1_ABCD-MPROC-T1_20161118113440.tgz
NDAR

NDARINV0/NDARINV0CCEN5K2_baselineYear1Arm1_ABCD-MPROC-T1_20180515121909.tgz
NDARINV0/NDARINV0GYC1NWR_baselineYear1Arm1_ABCD-MPROC-T1_20181005135701.tgz
NDARINV0/NDARINV03BDCNWM_baselineYear1Arm1_ABCD-MPROC-DTI_20170423115846.tgz
NDARINV0/NDARINV0FBX6KWX_baselineYear1Arm1_ABCD-MPROC-T1_20170803103716.tgz
NDARINV0/NDARINV0N289PCW_baselineYear1Arm1_ABCD-MPROC-T1_20171022105436.tgz
NDARINV0/NDARINV0AF2TMR4_baselineYear1Arm1_ABCD-MPROC-T1_20180917155345.tgz
NDARINV0/NDARINV0A86UD86_baselineYear1Arm1_ABCD-MPROC-T1_20180616101243.tgz
NDARINV0/NDARINV0ZTJ2432_baselineYear1Arm1_ABCD-MPROC-T1_20170509185057.tgz
NDARINV0/NDARINV022ZVCT8_baselineYear1Arm1_ABCD-MPROC-DTI_20171216092346.tgz
NDARINV0/NDARINV0NLL2KYE_baselineYear1Arm1_ABCD-MPROC-T1_20170319114428.tgz
NDARINV0/NDARINV02UVMTY7_baselineYear1Arm1_ABCD-MPROC-T1_20171007133904.tgz
NDARINV0/NDARINV0GWJ7GFF_baselineYear1Arm1_ABCD-MPROC-T1_20180301161502.tgz
NDARINV0/NDARINV0AAKGYA2_baselineYear1Arm1_ABCD-MPROC-T1_20180318143021.tgz
NDARINV0/N

NDARINV0/NDARINV0F82C6R8_baselineYear1Arm1_ABCD-MPROC-DTI_20170805103043.tgz
NDARINV0/NDARINV03XVEBPM_baselineYear1Arm1_ABCD-MPROC-DTI_20170520091053.tgz
NDARINV0/NDARINV0B3PHZ2W_baselineYear1Arm1_ABCD-MPROC-DTI_20180312125029.tgz
NDARINV0/NDARINV05WK8AN7_baselineYear1Arm1_ABCD-MPROC-DTI_20170508190351.tgz
NDARINV0/NDARINV0YLLNXRL_baselineYear1Arm1_ABCD-MPROC-DTI_20171020160040.tgz
NDARINV0/NDARINV0BVP2PTD_baselineYear1Arm1_ABCD-MPROC-DTI_20171202154758.tgz
NDARINV0/NDARINV0KYKANZ6_baselineYear1Arm1_ABCD-MPROC-DTI_20180725104001.tgz
NDARINV0/NDARINV0VBJ9TP2_baselineYear1Arm1_ABCD-MPROC-DTI_20170722135028.tgz
NDARINV0/NDARINV00LH735Y_baselineYear1Arm1_ABCD-MPROC-DTI_20180130173951.tgz
NDARINV0/NDARINV0E1RBR9J_baselineYear1Arm1_ABCD-MPROC-DTI_20180217133612.tgz
NDARINV0/NDARINV0L7XB3AW_baselineYear1Arm1_ABCD-MPROC-DTI_20170606144539.tgz
NDARINV0/NDARINV00NPMHND_baselineYear1Arm1_ABCD-MPROC-DTI_20170404165046.tgz
NDARINV0/NDARINV0RKF6F11_baselineYear1Arm1_ABCD-MPROC-T1_20170729124633.tgz


NDARINV0/NDARINV08AT7TLH_baselineYear1Arm1_ABCD-MPROC-DTI_20170206155058.tgz
NDARINV0/NDARINV0DYF4WPG_baselineYear1Arm1_ABCD-MPROC-DTI_20181003145347.tgz
NDARINV0/NDARINV0Z5FF0JL_baselineYear1Arm1_ABCD-MPROC-DTI_20170927152020.tgz
NDARINV0/NDARINV03XVEBPM_baselineYear1Arm1_ABCD-MPROC-T1_20170520084902.tgz
NDARINV0/NDARINV0ZPPK2ML_baselineYear1Arm1_ABCD-MPROC-T1_20180928132827.tgz
NDARINV0/NDARINV0A86UD86_baselineYear1Arm1_ABCD-MPROC-DTI_20180616103023.tgz
NDARINV0/NDARINV0A4P0LWM_baselineYear1Arm1_ABCD-MPROC-T1_20180310110114.tgz
NDARINV0/NDARINV0L7XB3AW_baselineYear1Arm1_ABCD-MPROC-T1_20170606141922.tgz
NDARINV0/NDARINV0LN1KD13_baselineYear1Arm1_ABCD-MPROC-DTI_20170920162056.tgz
NDARINV0/NDARINV0K37T6VD_baselineYear1Arm1_ABCD-MPROC-T1_20180109124123.tgz
NDARINV0/NDARINV0PDYPPJ7_baselineYear1Arm1_ABCD-MPROC-T1_20180104130708.tgz
NDARINV0/NDARINV0YGRZTUF_baselineYear1Arm1_ABCD-MPROC-T1_20180410144012.tgz
NDARINV0/NDARINV02WP3TP6_baselineYear1Arm1_ABCD-MPROC-T1_20171209094707.tgz
NDARINV

In [ ]:
last_member_names

In [ ]:
print(file_to_untar)
tar.extractall(members = file_to_untar, path = args.save_dir) #extract files


for i,member in enumerate(last_member_names):
    #raise ValueError("muyst be changed")
    upper_dir = os.path.split(member.name)[0] #get uppder dir name
    
    [shutil.move(file, args.save_dir) for file in glob.glob(os.path.join(args.save_dir, f"{upper_dir}/*"))] #move them to the actual directory I wanted

In [ ]:
os.chdir(args.save_dir)

#unzip the .tgz files
for file in os.listdir(args.save_dir):
    print(file)
    if "tgz" in file: #i.e. if tgz
        subprocess.run(f'tar zxvf {file}', shell = True)
        subprocess.run(f'rm {file}', shell = True)
    elif not "sub-" in file and "dataset_description.json" != file: #i.e. 이미 unzip된 sub-파일이 아니라면, 또는 dataset_description이 아니라면 
        shutil.rmtree(file) #remove the nuisance stuff


In [ ]:
raise ValueError("hi")

In [ ]:
import os 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import pdb
from pathlib import Path

T2_dir = Path("/work2/08834/tg881334/stampede2/ABCD_preproc/txt_files/T2/")
T1_DWI_dir = Path("/work2/08834/tg881334/stampede2/ABCD_preproc/txt_files/T1_DWI/")

In [ ]:
os.listdir(T2_dir)

In [ ]:
pd.read_csv(T2_dir / "dataset_collection.txt", sep = "\t")
pd.read_csv(T2_dir / "guid_pseudoguid.txt", sep = "\t")
pd.read_csv(T2_dir / "md5_values.txt", sep = "\t")
pd.read_csv(T2_dir / "fmriresults01.txt", sep = "\t") #this is what should be used
pd.read_csv(T2_dir / "package_info.txt", sep = "\t") #this is what should be used

# 0. actually reading the thing 

In [ ]:
T1_DWI_meta = pd.read_csv(T1_DWI_dir / "fmriresults01.txt", sep = "\t", header = 0 )
T2_meta = pd.read_csv(T2_dir / "fmriresults01.txt", sep = "\t", header = 0 ) 


In [ ]:
##removing the first layer because they won't be used (just extra descriptions)
descriptions = T2_meta.iloc[0]
T2_meta = T2_meta.drop(index = 0)
T1_DWI_meta = T1_DWI_meta.drop(index=0)


##seperating T1_DWI_meta into T1 and DWI 
T1_meta = T1_DWI_meta[T1_DWI_meta.scan_type == 'MR structural (T1)']
DWI_meta = T1_DWI_meta[T1_DWI_meta.scan_type == 'multi-shell DTI']

In [ ]:
##just testing

print(descriptions[0:5].values)
print(T1_meta.shape, T2_meta.shape, DWI_meta.shape)
T1_meta.head()

In [ ]:
##참조
#check/proof that we're only dealing with QC-ed subjects
(T1_meta.qc_outcome != "pass").sum() + (T2_meta.qc_outcome != "pass").sum() + (DWI_meta.qc_outcome != "pass").sum()

# 1. Merging the Datasets

## 1.1 Finding common subjects (which will be used)

# 잠만 : step0에서 그 release별로 구분하는 것도 해야함! (우리는 releae4 만 할것...)
# 따라서 밑에서 overlapping하는 사람들만 빼내는 것도, release4만 빼낸 후에 overlapping을 빼야할듯

In [ ]:
##defining the subject sets
T2_sub_set = set((T2_meta['subjectkey']).values)
T1_sub_set = set((T1_meta['subjectkey']).values)
DWI_sub_set = set((DWI_meta['subjectkey']).values)
print(len(T1_sub_set), len(T2_sub_set), len(DWI_sub_set))

In [ ]:
##getting intersection
total_set = (T2_sub_set.intersection(T1_sub_set)).intersection(DWI_sub_set)
print(len(total_set))

In [ ]:
meta_list = [T1_meta, T2_meta, DWI_meta]

meta = meta_list[0]

haha = meta.loc[meta['subjectkey'].isin(total_set)]
len(haha)
#for meta in meta_list:
    

In [ ]:
set(T1_DWI_meta.scan_type.values)

In [ ]:
T1_meta = T1_DWI_meta[T1_DWI_meta.scan_type == 'MR structural (T1)']
DWI_meta = T1_DWI_meta[T1_DWI_meta.scan_type == 'multi-shell DTI']

In [ ]:
#T2_meta.head()
T1_DWI_meta.head()

In [ ]:
#T1_DWI_meta.sort_values('subjectkey')
T1_DWI_sub_set = set((T1_DWI_meta['subjectkey'][1:]).values)
T2_sub_set = set((T2_meta['subjectkey'][1:]).values)

common = T1_DWI_sub_set.intersection(T2_sub_set)
T1_only = T1_DWI_sub_set.intersection(T2_sub_set)
T2_only = 
len(common)

In [ ]:
T2_meta.index #

In [ ]:
T2_meta.interview_age.head()

In [ ]:
T2_meta[(T2_meta['subjectkey'] != T2_meta['src_subject_id'])]


In [ ]:
T1_DWI_meta[(T1_DWI_meta['subjectkey'] != T1_DWI_meta['src_subject_id'])]

In [ ]:
T2_meta.shape

In [ ]:
T2_meta['subjectkey'][::1000]

In [ ]:
T1_DWI_meta['subjectkey'][::1000]

In [ ]:
(T2_meta['qc_outcome'] == "pass").sum()

In [ ]:
set(T2_meta['qc_outcome'].values)